In [1]:
import websocket
import json
import zlib
import threading
from threading import Thread
from threading import Lock
import queue
import time
import sqlite3
import pandas as pd
import asyncio
import nest_asyncio
import websockets

In [ ]:
from pybit.unified_trading import WebSocket
from time import sleep
ws = WebSocket(
    testnet=True,
    channel_type="linear",
)
def handle_message(message):
    print(message)
symbol_list = ['BTC','ETH']
# [Symbol for Symbol in symbol_list] 
Symbol = 'ETH'
ws.trade_stream(
    symbol= Symbol + "USDT",
    callback=handle_message
)
# while True:
#     sleep(1)